## Sim Demo

### Imports

In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from imports import *

In [68]:
# overall imports
import importlib
import data

import sim.sim 
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run, open_pickled_results



#### Check job specs

In [69]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 21.7%
RAM Usage: 4.4%
Available RAM: 963.1G
Total RAM: 1007.0G
52.4G


In [70]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

XGBoost version: 2.0.3
cupy version: 13.1.0


In [71]:
GPUtil.getAvailable()
# if a number is seen a GPU is available

[0]

In [72]:
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

GPU found 0


In [73]:
GPUtil.showUtilization()


| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |


In [74]:
torch.cuda.is_available()


True

#### Simulation tests <a id="sims"></a>

In [ ]:
single_sim_run(
              cv_type='random',
              random_seed=42,
              model_type='dynamic_nn',
              feature_type=[{'transcriptome': 'PCA'}],
              connectome_target='FC',
              use_gpu=True,
              use_shared_regions=True,
              test_shared_regions=False,
              save_sim=False,
              search_method=('grid', 'mse'),
              track_wandb=True
              )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


feature_name transcriptome
processing_type PCA
X shape (114, 34)

 Test fold num: 1
(12070, 68) (12070,) (812, 68) (812,)
SEARCH METHOD ('grid', 'mse')
2
3
4
Create sweep with ID: 9vdx40iv
Sweep URL: https://wandb.ai/alexander-ratzan-new-york-university/gx2conn/sweeps/9vdx40iv
sweep id on outer call: 9vdx40iv


wandb: Agent Starting Run: hccf6k49 with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0
wandb: 	epochs: 10
wandb: 	hidden_dims: [512, 256, 128]
wandb: 	input_dim: 68
wandb: 	learning_rate: 0.0003
wandb: 	symmetry_weight: 0.1
wandb: 	weight_decay: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Processing inner fold 0
Epoch 10/10, Train Loss: 0.0233, Val Loss: 0.0258
Processing inner fold 1
Epoch 10/10, Train Loss: 0.0229, Val Loss: 0.0343
Processing inner fold 2
Epoch 10/10, Train Loss: 0.0258, Val Loss: 0.0148


fold0_epoch,▁▂▃▃▄▅▆▆▇█
fold0_train_loss,█▃▂▂▁▁▁▁▁▁
fold0_train_pearson,▁▄▅▆▇▇▇███
fold0_val_loss,█▃▁▂▁▂▁▁▁▁
fold0_val_pearson,▃▁▅▃▄▅▆▅█▆
fold1_epoch,▁▂▃▃▄▅▆▆▇█
fold1_train_loss,█▂▂▁▁▁▁▁▁▁
fold1_train_pearson,▁▄▅▆▇▇▇███
fold1_val_loss,█▃▄▃▁▂▁▁▁▁
fold1_val_pearson,▁▆▅▅▅▆██▇▇
fold2_epoch,▁▂▃▃▄▅▆▆▇█


...

wandb: Agent Starting Run: vlgb1rjj with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.1
wandb: 	epochs: 10
wandb: 	hidden_dims: [512, 256, 128]
wandb: 	input_dim: 68
wandb: 	learning_rate: 0.001
wandb: 	symmetry_weight: 0
wandb: 	weight_decay: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Processing inner fold 0
Epoch 10/10, Train Loss: 0.0158, Val Loss: 0.0254
Processing inner fold 1
Epoch 10/10, Train Loss: 0.0187, Val Loss: 0.0381
Processing inner fold 2
Epoch 10/10, Train Loss: 0.0174, Val Loss: 0.0146


fold0_epoch,▁▂▃▃▄▅▆▆▇█
fold0_train_loss,█▂▂▂▁▁▁▁▁▁
fold0_train_pearson,▁▄▅▆▆▇▇███
fold0_val_loss,▄▁▅▄▃▂▄█▄▄
fold0_val_pearson,▁▅▅▆▇▇▇▇██
fold1_epoch,▁▂▃▃▄▅▆▆▇█
fold1_train_loss,█▃▂▂▁▁▁▁▁▁
fold1_train_pearson,▁▃▄▆▆▇▇███
fold1_val_loss,█▁▁▁▂▂▂▄▂▃
fold1_val_pearson,▁▃▄▆▆▇█▆█▇
fold2_epoch,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: b8ucgyod with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.1
wandb: 	epochs: 10
wandb: 	hidden_dims: [256, 128]
wandb: 	input_dim: 68
wandb: 	learning_rate: 0.0001
wandb: 	symmetry_weight: 0
wandb: 	weight_decay: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Processing inner fold 0
Epoch 10/10, Train Loss: 0.0286, Val Loss: 0.0322
Processing inner fold 1
Epoch 10/10, Train Loss: 0.0305, Val Loss: 0.0568
Processing inner fold 2


In [34]:
single_sim_run(
              cv_type='random',
              random_seed=42,
              model_type='ridge',
              feature_type=[
                            {'structural': 'spectral_A_20'}
                            ],
              use_gpu=False,
              use_shared_regions=False,
              test_shared_regions=False,
              save_sim=True,
              connectome_target='SC',
              search_method=('grid', 'mse')
              )

computing eig of laplacian
computing eig of adjacency
feature_type [{'structural': 'spectral_A_20'}]
feature_name structural
processing_type spectral_A_20
features ['structural_spectral_A_20']
feature structural_spectral_A_20
X shape (114, 20)

 Test fold num: 1
(7140, 40) (7140,) (812, 40) (812,)
SEARCH METHOD ('grid', 'mse')
2
3
4
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END .............alpha=0, solver=auto;, score=-8.332 total time=   0.0s
[CV 2/3] END .............alpha=0, solver=auto;, score=-9.799 total time=   0.0s
[CV 3/3] END ............alpha=0, solver=auto;, score=-10.367 total time=   0.0s
[CV 1/3] END .........alpha=0.001, solver=auto;, score=-8.332 total time=   0.0s
[CV 2/3] END .........alpha=0.001, solver=auto;, score=-9.800 total time=   0.0s
[CV 3/3] END ........alpha=0.001, solver=auto;, score=-10.368 total time=   0.0s
[CV 1/3] END ..........alpha=0.01, solver=auto;, score=-8.330 total time=   0.0s
[CV 2/3] END ..........alpha=0.01, sol

[[{'model_parameters': {'alpha': 0,
    'copy_X': True,
    'fit_intercept': True,
    'max_iter': None,
    'positive': False,
    'random_state': None,
    'solver': 'auto',
    'tol': 0.0001},
   'train_metrics': {'mse': 9.171478346397917,
    'mae': 2.357787486870799,
    'r2': 0.039259094114764095,
    'pearson_corr': 0.31565541467463465,
    'connectome_corr': 0.2128484310373962,
    'connectome_r2': 0.03554164231750065,
    'geodesic_distance': 25.386686228006337},
   'best_val_score': -9.49922680893964,
   'test_metrics': {'mse': 8.008470649591864,
    'mae': 2.075622864995832,
    'r2': 0.028827732071347107,
    'pearson_corr': 0.33946316440655133,
    'connectome_corr': 0.23550330059145658,
    'connectome_r2': 0.01864952452719144,
    'geodesic_distance': 13.425229214523961},
   'y_true': array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
           0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
           9.9884,  9.9884,  4.5589,  4.5589, 

In [36]:
single_sim_run(
              cv_type='random',
              random_seed=42,
              model_type='ridge',
              feature_type=[{'transcriptome': None}, 
                            {'structural': 'spectral_A_20'}
                            ],
              use_gpu=False,
              use_shared_regions=False,
              test_shared_regions=False,
              save_sim=True,
              connectome_target='FC',
              search_method=('grid', 'mse')
              )

computing eig of laplacian
computing eig of adjacency
feature_type [{'transcriptome': None}, {'structural': 'spectral_A_20'}]
feature_name transcriptome
processing_type None
feature_name structural
processing_type spectral_A_20
features ['transcriptome', 'structural_spectral_A_20']
feature transcriptome
feature structural_spectral_A_20
X shape (114, 11073)

 Test fold num: 1
(7140, 22146) (7140,) (812, 22146) (812,)
SEARCH METHOD ('grid', 'mse')
2
3
4
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END .............alpha=0, solver=auto;, score=-0.026 total time=   3.2s
[CV 2/3] END .............alpha=0, solver=auto;, score=-0.028 total time=   3.1s
[CV 3/3] END .............alpha=0, solver=auto;, score=-0.041 total time=   3.2s
[CV 1/3] END .........alpha=0.001, solver=auto;, score=-0.026 total time=   1.3s
[CV 2/3] END .........alpha=0.001, solver=auto;, score=-0.028 total time=   1.3s
[CV 3/3] END .........alpha=0.001, solver=auto;, score=-0.034 total time=   1.3

[[{'model_parameters': {'alpha': 1000,
    'copy_X': True,
    'fit_intercept': True,
    'max_iter': None,
    'positive': False,
    'random_state': None,
    'solver': 'auto',
    'tol': 0.0001},
   'train_metrics': {'mse': 0.021032101193248658,
    'mae': 0.10500131429428843,
    'r2': -1.139266997025029,
    'pearson_corr': 0.6331792766809794,
    'connectome_corr': 0.4598820738973434,
    'connectome_r2': -1.2561298441624122,
    'geodesic_distance': 13.408910811166907},
   'best_val_score': -0.02714152611248866,
   'test_metrics': {'mse': 0.021574909319022582,
    'mae': 0.10984827356812175,
    'r2': -0.8116136167331731,
    'pearson_corr': 0.5387020135228298,
    'connectome_corr': 0.4097680692386645,
    'connectome_r2': -1.0358187236230398,
    'geodesic_distance': 6.421698352595828},
   'y_true': array([ 0.25718  ,  0.25718  ,  0.21614  ,  0.21614  ,  0.17879  ,
           0.17879  ,  0.20003  ,  0.20003  ,  0.18572  ,  0.18572  ,
           0.25026  ,  0.25026  ,  0.11669 